<a href="https://colab.research.google.com/github/arimbawa/Aplikasi-IoT/blob/main/Minggu_7_Keamanan_IoT_(HTTP_%26_MQTT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Minggu ke-7: Keamanan IoT (HTTP & MQTT)**
**Tujuan Pembelajaran:**

1. Mahasiswa memahami risiko keamanan pada komunikasi IoT.
2. Mahasiswa dapat menerapkan HTTPS (SSL/TLS) pada komunikasi HTTP.
3. Mahasiswa dapat mengamankan MQTT dengan username-password authentication.
4. Mahasiswa mengetahui praktik dasar secure coding dalam aplikasi IoT berbasis Flask & MQTT.

# Praktik di Google Colab

# 1. Install modul

In [1]:
!pip install flask pyngrok paho-mqtt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 5.3 MB/s eta 0:00:00


# 2. Flask dengan Token Authentication

In [2]:
# Konfigurasi Ngrok dengan token Agus
from pyngrok import ngrok

NGROK_TOKEN = "32tnIljunaDDrFMWVCD2784cERi_2tHAN1MwNfRM9Ug5omaq5"
ngrok.set_auth_token(NGROK_TOKEN)

In [3]:
from flask import Flask, request, jsonify
from pyngrok import ngrok

app = Flask(__name__)

# Token sederhana
API_TOKEN = "iotsecure123"

@app.route('/secure-data', methods=['POST'])
def secure_data():
    token = request.headers.get("Authorization")
    if token != f"Bearer {API_TOKEN}":
        return jsonify({"error": "Unauthorized"}), 401

    data = request.json
    return jsonify({"status": "success", "data_received": data}), 200

# Jalankan ngrok
public_url = ngrok.connect(5000)
print(" * Ngrok URL (secure):", public_url)

app.run(port=5000)

 * Ngrok URL (secure): NgrokTunnel: "https://dae2a706d53b.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [02/Oct/2025 12:51:34] "POST /secure-data HTTP/1.1" 200 -


**Cara uji:**

In [ ]:
import requests

url = "<NGROK_URL>/secure-data"
headers = {"Authorization": "Bearer iotsecure123"}
data = {"temperature": 25, "humidity": 60}

r = requests.post(url, json=data, headers=headers)
print(r.text)

Jika tanpa header Authorization → 401 Unauthorized.

# 3. MQTT dengan Username & Password (HiveMQ broker publik)

In [ ]:
import paho.mqtt.client as mqtt
import random, time, json

# HiveMQ public broker with auth (contoh)
BROKER = "broker.hivemq.com"
PORT = 1883
TOPIC = "unram/iot/secure"
USERNAME = "iot_user"
PASSWORD = "iot_pass"

def on_connect(client, userdata, flags, rc):
    print("Connected with result code", rc)

def on_message(client, userdata, msg):
    print(f"Message received: {msg.payload.decode()}")

client = mqtt.Client()
client.username_pw_set(USERNAME, PASSWORD)  # Auth
client.on_connect = on_connect
client.on_message = on_message

client.connect(BROKER, PORT, 60)
client.loop_start()

# Publish data sensor dengan autentikasi
for i in range(50):
    payload = {"temperature": random.randint(20,30),
               "humidity": random.randint(60,80)}
    client.publish(TOPIC, json.dumps(payload))
    print("Published:", payload)
    time.sleep(3)

In [11]:
import paho.mqtt.client as mqtt
import time
import json
import random

# MQTT Broker details
BROKER = "9efe7c37a1504f69b9b9267672364909.s1.eu.hivemq.cloud" # Replace with your broker address if different
PORT = 8883
USERNAME = "iot_user" # Replace with your username if required
PASSWORD = "IoT_pass123" # Replace with your password if required
TOPIC = "unram/iot/secure"

def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected to MQTT Broker!")
    else:
        print("Failed to connect, return code %d\n", rc)

client = mqtt.Client()
# Uncomment if authentication is required
client.username_pw_set(USERNAME, PASSWORD)
client.on_connect = on_connect

client.connect(BROKER, PORT)
client.loop_start()

try:
    while True:
        # Your message format
        payload = {"temperature": random.randint(20,30),
                   "humidity": random.randint(60,80)}

        # Serialize the dictionary to a JSON string and then encode to bytes
        message = json.dumps(payload)
        client.publish(TOPIC, message)

        print(f"Published message: {message}")
        time.sleep(5) # Publish every 5 seconds

except KeyboardInterrupt:
    print("Disconnecting from MQTT Broker.")
    client.loop_stop()
    client.disconnect()

/tmp/ipython-input-706157842.py:19: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


Published message: {"temperature": 27, "humidity": 72}
Published message: {"temperature": 30, "humidity": 77}
Published message: {"temperature": 23, "humidity": 64}
Published message: {"temperature": 21, "humidity": 69}
Published message: {"temperature": 27, "humidity": 61}
Published message: {"temperature": 23, "humidity": 61}
Published message: {"temperature": 20, "humidity": 79}
Published message: {"temperature": 30, "humidity": 61}
Published message: {"temperature": 21, "humidity": 65}
Published message: {"temperature": 25, "humidity": 68}
Published message: {"temperature": 27, "humidity": 73}
Published message: {"temperature": 23, "humidity": 75}
Published message: {"temperature": 22, "humidity": 76}
Published message: {"temperature": 23, "humidity": 72}
Published message: {"temperature": 25, "humidity": 65}
Published message: {"temperature": 26, "humidity": 73}
Published message: {"temperature": 28, "humidity": 62}
Published message: {"temperature": 26, "humidity": 60}
Published 

In [ ]:
# Subscribe to the topic
client.subscribe(TOPIC)

# Keep the client running to process incoming messages
# You might want to run this in a separate thread or process in a real application
try:
    client.loop_forever()
except KeyboardInterrupt:
    print("Disconnecting from MQTT broker.")
    client.disconnect()

# Tugas Minggu ke-7

1. Ubah server Flask agar menggunakan HTTPS (bisa dengan sertifikat self-signed atau pyOpenSSL).
2. Buat client IoT yang hanya bisa mengirim data dengan API key valid.
3. Konfigurasi MQTT client agar gagal connect jika username/password salah.
4. Buat laporan singkat: bandingkan komunikasi tanpa keamanan vs dengan keamanan (HTTP/MQTT).T